In [6]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import plotly.express as px
import pandas as pd

pd.set_option('display.float_format', lambda x: f'{x :.2f}')
from get_dataframes import GetDataframes
import hjson

import matplotlib.pyplot as plt
import explo_data_analysis.eda_movies as eda

with open("config.hjson") as fp:
    config = hjson.load(fp)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
datas = GetDataframes(config)
link = "directors_movies"
df = datas.get_dataframes(
    link,
)

2023-10-30 10:46:55 INFO     Parquet loaded ! Importing directors...
2023-10-30 10:46:58 INFO     Parquet loaded ! Importing persons...
2023-10-30 10:47:14 INFO     File movies.parquet not found. Creation...
2023-10-30 10:47:14 INFO     Polars loaded ! Importing title_basics...
2023-10-30 10:47:16 INFO     Cleaning porn movies...
2023-10-30 10:47:16 INFO     Cleaned : 9235 rows
2023-10-30 10:47:16 INFO     Polars loaded ! Importing title_ratings...
2023-10-30 10:47:17 INFO     Joining movies dataframes...
2023-10-30 10:47:17 INFO     Renaming movies columns...
2023-10-30 10:47:17 INFO     Fixing N values...
2023-10-30 10:47:17 INFO     Pandas loaded ! Importing tmdb_full...
2023-10-30 10:47:21 INFO     Cleaned NaN Value : 108679
2023-10-30 10:47:21 INFO     Length dataframe merged with tmdb : 185564
2023-10-30 10:47:21 INFO     Length dataframe merged cleaned : 149505
2023-10-30 10:47:25 INFO     Pandas loaded ! Importing title_akas...
2023-10-30 10:48:04 INFO     Merging tmdb and akas

In [10]:
print(df.head().to_markdown())

|    | titre_id   | titre_str           |   titre_date_sortie |   titre_duree | titre_genres           |   rating_avg |   rating_votes | original_language   | original_title                       |   popularity | production_countries   |   revenue | spoken_languages   | status   | region   | cuts      | person_id   | person_name     |   person_birthdate | person_job   | person_role   | person_film                                       |   person_index |
|---:|:-----------|:--------------------|--------------------:|--------------:|:-----------------------|-------------:|---------------:|:--------------------|:-------------------------------------|-------------:|:-----------------------|----------:|:-------------------|:---------|:---------|:----------|:------------|:----------------|-------------------:|:-------------|:--------------|:--------------------------------------------------|---------------:|
|  0 | tt0009128  | The Great Adventure |                1918 |            50 | ['Dr

In [9]:
df[df.titre_str.str.contains("Terminator")]

,titre_id,titre_str,titre_date_sortie,titre_duree,titre_genres,rating_avg,rating_votes,original_language,original_title,popularity,...,status,region,cuts,person_id,person_name,person_birthdate,person_job,person_role,person_film,person_index
20732,tt0199849,Ninja Terminator,1986,88,"[Action, Adventure]",4.40,1357,en,Ninja Terminator,5.46,...,Released,FR,1980-1989,nm0061792,Godfrey Ho,1948,director,[Unknown],"[tt0399762, tt0201983, tt0165138, tt0199845]",5
22408,tt0102079,Angel Terminators II,1992,93,"[Action, Crime, Thriller]",6.70,205,cn,火種,2.34,...,Released,FR,1990-1999,nm0490541,Chun-Ku Lu,1946,director,[Unknown],"[tt0071323, tt12229444, tt0099416, tt0078393]",6
22409,tt0102079,Angel Terminators II,1992,93,"[Action, Crime, Thriller]",6.70,205,cn,火種,2.34,...,Released,FR,1990-1999,nm0150991,Lau Chan,0,director,[Unknown],"[tt0184190, tt0184443, tt0102079, tt2349693]",5
22914,tt0100531,Russian Terminator,1989,87,[Action],2.80,476,en,The Russian Ninja,1.49,...,Released,FR,1980-1989,nm0375093,Mats Helge Olsson,1953,director,[Unknown],"[tt0100531, tt0095141, tt0101156, tt0097801]",5
22915,tt0100531,Russian Terminator,1989,87,[Action],2.80,476,en,The Russian Ninja,1.49,...,Released,FR,1980-1989,nm0632152,Anders Nilsson,1963,director,[Unknown],"[tt0782700, tt0220623, tt0198837, tt0313724]",6
25672,tt0088247,The Terminator,1984,107,"[Action, Sci-Fi]",8.10,902644,en,The Terminator,49.42,...,Released,FR,1980-1989,nm0000116,James Cameron,1954,director,[Unknown],"[tt0499549, tt0090605, tt1630029, tt0120338]",5
25675,tt0103064,Terminator 2: Judgment Day,1991,137,"[Action, Sci-Fi]",8.60,1145262,en,Terminator 2: Judgment Day,50.31,...,Released,FR,1990-1999,nm0000116,James Cameron,1954,director,[Unknown],"[tt0499549, tt0090605, tt1630029, tt0120338]",5
28907,tt0095483,Lady Terminator,1989,82,"[Action, Drama, Fantasy]",5.50,1941,id,Pembalasan Ratu Pantai Selatan,3.04,...,Released,FR,1980-1989,nm0229083,H. Tjut Djalil,1932,director,[Unknown],"[tt0280953, tt0346129, tt0095483, tt0272063]",5
29431,tt1340138,Terminator Genisys,2015,126,"[Action, Adventure, Sci-Fi]",6.30,288408,en,Terminator Genisys,60.63,...,Released,FR,2010-2019,nm0851930,Alan Taylor,1965,director,[Unknown],"[tt1981115, tt0944947, tt1340138, tt0282768]",5
29564,tt0181852,Terminator 3: Rise of the Machines,2003,109,"[Action, Sci-Fi]",6.30,412807,en,Terminator 3: Rise of the Machines,40.15,...,Released,FR,2000-2009,nm0609236,Jonathan Mostow,1961,director,[Unknown],"[tt0181852, tt0118771, tt0986263, tt0141926]",5
